In [ ]:
# https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_mystem.py
# https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb

In [1]:
import sys
import requests

import gensim
from pymystem3 import Mystem

# Preprocessing

In [6]:
mapping_url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mystem2upos = {}
r = requests.get(mapping_url, stream=True)
for pair in r.text.split('\n'):
    pair = pair.split()
    if len(pair) > 1:
        mystem2upos[pair[0]] = pair[1]

class Preprocessor:
    def __init__(self, mapping):
        self.m = Mystem()
        self.mapping = mapping
        
    def process(self, text, postags=True):
        processed = self.m.analyze(text)
        tagged = []
        for w in processed:
            try:
                lemma = w["analysis"][0]["lex"].lower().strip()
                pos = w["analysis"][0]["gr"].split(',')[0]
                pos = pos.split('=')[0].strip()
                pos = self.mapping.get(pos, 'X')
                tagged.append(lemma.lower() + '_' + pos)
            except KeyError:
                continue
        if not postags:
            tagged = [t.split('_')[0] for t in tagged]
        return tagged

phrases_processor = Preprocessor(mystem2upos)

In [9]:
phrases_processor.process(',"Купил для магнитолы в машину, пользуюсь уже 3-ю неделю, никаких нареканий нет, не тормозит, при подключении к магнитоле/компьютеру начинает работать сразу же и без задержек. В общем все в порядке, работает как часы."', postags=False)

['купить',
 'для',
 'магнитола',
 'в',
 'машина',
 'пользоваться',
 'уже',
 'ю',
 'неделя',
 'никакой',
 'нарекание',
 'нет',
 'не',
 'тормозить',
 'при',
 'подключение',
 'к',
 'магнитола',
 'компьютер',
 'начинать',
 'работать',
 'сразу',
 'же',
 'и',
 'без',
 'задержка',
 'в',
 'общий',
 'весь',
 'в',
 'порядок',
 'работать',
 'как',
 'часы']

# Word2Vec model

In [6]:
# You can download pretrained model from http://rusvectores.org/ru/models/

model_path = '/mnt/disk/datasets/word2vec_ru/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=False)

In [26]:
model.most_similar('платить_VERB')

[('уплатить_VERB', 0.7811346650123596),
 ('доплачивать_VERB', 0.7447771430015564),
 ('выплачивать_VERB', 0.7416616082191467),
 ('уплачивать_VERB', 0.736369252204895),
 ('оплачивать_VERB', 0.7315130829811096),
 ('заплатить_VERB', 0.7277652025222778),
 ('приплачивать_VERB', 0.6747636795043945),
 ('задолжать_VERB', 0.664445698261261),
 ('расплачиваться_VERB', 0.6592860221862793),
 ('плащать_VERB', 0.654596209526062)]

In [35]:
word_vector = model['платить_VERB']
word_vector.shape

(300,)